# Finalizing a Prediction Strategy

In previous notebooks, we explored different data preprocessing techniques, feature engineering, and testing various sklearn models for predicting NBA games.

Now, we will attempt to finalize a prediction strategy using the best performing models and other techniques.

In [1]:
import pandas as pd
from sklearn import metrics

## Read the dataset

In [2]:
path_to_data = "../../data/processed/processed_team_dataset.csv"

df = pd.read_csv(path_to_data, index_col=0)
df

,date,season,team,team_opp,team_elo_before,team_opp_elo_before,team_expected_win_probability,team_point_diff_proj,home,home_opp,...,drb%_max_allowed_last_10_opp,trb%_max_allowed_last_10_opp,ast%_max_allowed_last_10_opp,stl%_max_allowed_last_10_opp,blk%_max_allowed_last_10_opp,tov%_max_allowed_last_10_opp,usg%_max_allowed_last_10_opp,ortg_max_allowed_last_10_opp,drtg_max_allowed_last_10_opp,total_allowed_last_10_opp
0,2015-10-29,2016,NYK,ATL,1356.009483,1541.442237,0.379472,-3.051170,1,0,...,41.20,24.80,35.60,3.20,4.70,33.30,23.60,132.0,104.0,106.0
1,2015-10-29,2016,ATL,NYK,1541.442237,1356.009483,0.620528,3.051170,0,1,...,20.50,20.30,34.90,3.40,6.80,50.00,32.50,140.0,137.0,97.0
2,2015-10-29,2016,LAC,DAL,1652.435518,1559.753058,0.751974,6.881516,1,0,...,38.50,28.10,32.20,4.20,4.40,66.70,27.90,162.0,114.0,95.0
3,2015-10-29,2016,IND,MEM,1499.317744,1557.619147,0.559723,1.489236,1,0,...,34.50,23.50,30.60,8.40,12.20,100.00,28.00,149.0,89.0,106.0
4,2015-10-29,2016,MEM,IND,1557.619147,1499.317744,0.440277,-1.489236,0,1,...,42.50,24.80,30.20,2.60,4.80,100.00,29.70,134.0,105.0,106.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21751,2024-02-15,2024,MIL,MEM,1539.547927,1410.179450,0.542164,1.048874,0,1,...,35.17,26.01,46.02,5.42,9.31,36.66,30.13,176.2,115.0,114.4
21752,2024-02-15,2024,MEM,MIL,1410.179450,1539.547927,0.457836,-1.048874,1,0,...,40.78,23.44,43.28,5.62,7.04,37.04,43.73,175.3,122.7,113.4
21753,2024-02-15,2024,MIN,POR,1676.024592,1333.586609,0.801482,8.658499,0,1,...,33.23,22.67,39.22,13.52,8.59,36.08,38.80,180.1,124.0,114.5
21754,2024-02-15,2024,POR,MIN,1333.586609,1676.024592,0.198518,-8.658499,1,0,...,38.21,23.52,44.58,4.41,9.39,45.89,35.88,191.6,129.7,103.6


In [3]:
# Check that we have no null values

# Finding the number of missing values in each column
missing_values = df.isnull().sum()

# Printing the number of missing values in each column
print(sum(missing_values))

0


## Separate relevant features from the labels

In [4]:
# drop these columns (most of the are non_numeric or are not useful for machine learning)
drop_columns = ["date", "season", "team", "team_opp", "won"]

selected_columns = df.columns[~df.columns.isin(drop_columns)]

# Selected features
features_df = df[selected_columns]
features_df

,team_elo_before,team_opp_elo_before,team_expected_win_probability,team_point_diff_proj,home,home_opp,fg_last_10,fga_last_10,fg%_last_10,3p_last_10,...,drb%_max_allowed_last_10_opp,trb%_max_allowed_last_10_opp,ast%_max_allowed_last_10_opp,stl%_max_allowed_last_10_opp,blk%_max_allowed_last_10_opp,tov%_max_allowed_last_10_opp,usg%_max_allowed_last_10_opp,ortg_max_allowed_last_10_opp,drtg_max_allowed_last_10_opp,total_allowed_last_10_opp
0,1356.009483,1541.442237,0.379472,-3.051170,1,0,42.0,93.0,0.4520,9.0,...,41.20,24.80,35.60,3.20,4.70,33.30,23.60,132.0,104.0,106.0
1,1541.442237,1356.009483,0.620528,3.051170,0,1,37.0,82.0,0.4510,8.0,...,20.50,20.30,34.90,3.40,6.80,50.00,32.50,140.0,137.0,97.0
2,1652.435518,1559.753058,0.751974,6.881516,1,0,42.0,80.0,0.5250,6.0,...,38.50,28.10,32.20,4.20,4.40,66.70,27.90,162.0,114.0,95.0
3,1499.317744,1557.619147,0.559723,1.489236,1,0,32.0,86.0,0.3720,9.0,...,34.50,23.50,30.60,8.40,12.20,100.00,28.00,149.0,89.0,106.0
4,1557.619147,1499.317744,0.440277,-1.489236,0,1,29.0,82.0,0.3540,2.0,...,42.50,24.80,30.20,2.60,4.80,100.00,29.70,134.0,105.0,106.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21751,1539.547927,1410.179450,0.542164,1.048874,0,1,42.0,87.1,0.4819,15.0,...,35.17,26.01,46.02,5.42,9.31,36.66,30.13,176.2,115.0,114.4
21752,1410.179450,1539.547927,0.457836,-1.048874,1,0,37.4,85.4,0.4387,14.3,...,40.78,23.44,43.28,5.62,7.04,37.04,43.73,175.3,122.7,113.4
21753,1676.024592,1333.586609,0.801482,8.658499,0,1,42.0,83.8,0.5023,13.9,...,33.23,22.67,39.22,13.52,8.59,36.08,38.80,180.1,124.0,114.5
21754,1333.586609,1676.024592,0.198518,-8.658499,1,0,40.4,86.0,0.4683,10.4,...,38.21,23.52,44.58,4.41,9.39,45.89,35.88,191.6,129.7,103.6


In [5]:
# Label we want to predict
label = df["won"]
label

0        False
1         True
2         True
3        False
4         True
         ...  
21751    False
21752     True
21753     True
21754    False
21755    False
Name: won, Length: 21756, dtype: bool

### Remember that elo is about 64.48% accurate at predicting winner of a game

In [6]:
projected_point_spread = df["team_point_diff_proj"]
projected_win_from_elo = projected_point_spread > 0

print(metrics.accuracy_score(label, projected_win_from_elo))

0.6485567199852914


## Scale data for Logistic Regression

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaled_features_df = pd.DataFrame(scaler.fit_transform(features_df), columns=features_df.columns)
scaled_features_df

,team_elo_before,team_opp_elo_before,team_expected_win_probability,team_point_diff_proj,home,home_opp,fg_last_10,fga_last_10,fg%_last_10,3p_last_10,...,drb%_max_allowed_last_10_opp,trb%_max_allowed_last_10_opp,ast%_max_allowed_last_10_opp,stl%_max_allowed_last_10_opp,blk%_max_allowed_last_10_opp,tov%_max_allowed_last_10_opp,usg%_max_allowed_last_10_opp,ortg_max_allowed_last_10_opp,drtg_max_allowed_last_10_opp,total_allowed_last_10_opp
0,0.240976,0.517230,0.374891,0.438248,1.0,0.0,0.548387,0.454545,0.562044,0.391304,...,0.264080,0.135632,0.182741,0.062500,0.105856,0.218970,0.000000,0.120419,0.303571,0.447761
1,0.517230,0.240976,0.625109,0.561752,0.0,1.0,0.387097,0.204545,0.558394,0.347826,...,0.005006,0.083908,0.173858,0.071429,0.153153,0.414520,0.116492,0.162304,0.892857,0.313433
2,0.682585,0.544509,0.761551,0.639273,1.0,0.0,0.548387,0.159091,0.828467,0.260870,...,0.230288,0.173563,0.139594,0.107143,0.099099,0.610070,0.056283,0.277487,0.482143,0.283582
3,0.454473,0.541330,0.561992,0.530140,1.0,0.0,0.225806,0.295455,0.270073,0.391304,...,0.180225,0.120690,0.119289,0.294643,0.274775,1.000000,0.057592,0.209424,0.035714,0.447761
4,0.541330,0.454473,0.438008,0.469860,0.0,1.0,0.129032,0.204545,0.204380,0.086957,...,0.280350,0.135632,0.114213,0.035714,0.108108,1.000000,0.079843,0.130890,0.321429,0.447761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21751,0.514407,0.321677,0.543767,0.521228,0.0,1.0,0.548387,0.320455,0.671168,0.652174,...,0.188611,0.149540,0.314975,0.161607,0.209685,0.258314,0.085471,0.351832,0.500000,0.573134
21752,0.321677,0.514407,0.456233,0.478772,1.0,0.0,0.400000,0.281818,0.513504,0.621739,...,0.258824,0.120000,0.280203,0.170536,0.158559,0.262763,0.263482,0.347120,0.637500,0.558209
21753,0.717727,0.207571,0.812940,0.675237,0.0,1.0,0.548387,0.245455,0.745620,0.604348,...,0.164330,0.111149,0.228680,0.523214,0.193468,0.251522,0.198953,0.372251,0.660714,0.574627
21754,0.207571,0.717727,0.187060,0.324763,1.0,0.0,0.496774,0.295455,0.621533,0.452174,...,0.226658,0.120920,0.296701,0.116518,0.211486,0.366393,0.160733,0.432461,0.762500,0.411940


## Test Train Split

In [8]:
from sklearn.model_selection import train_test_split

test_ratio = 0.3
x_train_scaled, x_test_scaled, y_train_scaled, y_test_scaled = train_test_split(scaled_features_df, label, test_size = 0.3) # 70% data is training and 30% is for testing

x_train_scaled

,team_elo_before,team_opp_elo_before,team_expected_win_probability,team_point_diff_proj,home,home_opp,fg_last_10,fga_last_10,fg%_last_10,3p_last_10,...,drb%_max_allowed_last_10_opp,trb%_max_allowed_last_10_opp,ast%_max_allowed_last_10_opp,stl%_max_allowed_last_10_opp,blk%_max_allowed_last_10_opp,tov%_max_allowed_last_10_opp,usg%_max_allowed_last_10_opp,ortg_max_allowed_last_10_opp,drtg_max_allowed_last_10_opp,total_allowed_last_10_opp
11769,0.583621,0.656341,0.575915,0.536998,1.0,0.0,0.500000,0.400000,0.550000,0.478261,...,0.259700,0.145172,0.325254,0.109375,0.188288,0.202693,0.176178,0.409948,0.748214,0.504478
57,0.706698,0.181061,0.822642,0.682748,0.0,1.0,0.462366,0.340909,0.548662,0.405797,...,0.429912,0.413793,0.468909,0.209821,0.338964,0.173302,0.188482,0.578534,0.241071,0.440299
14113,0.695833,0.651282,0.396688,0.449334,0.0,1.0,0.606452,0.386364,0.697445,0.504348,...,0.244681,0.111494,0.201777,0.147321,0.168243,0.295082,0.126440,0.422513,0.760714,0.383582
3997,0.361154,0.458880,0.269802,0.380304,0.0,1.0,0.519355,0.343182,0.613869,0.360870,...,0.271214,0.254023,0.319797,0.145982,0.171847,0.331850,0.286387,0.368586,0.541071,0.432836
9399,0.591544,0.130991,0.929057,0.795733,1.0,0.0,0.538710,0.386364,0.598175,0.443478,...,0.276095,0.154713,0.304822,0.186161,0.177027,0.433138,0.181021,0.384293,0.469643,0.526866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,0.534728,0.577682,0.604848,0.551441,1.0,0.0,0.182796,0.219697,0.270073,0.333333,...,0.196496,0.086207,0.304146,0.300595,0.244745,0.217408,0.077661,0.403141,0.684524,0.532338
2438,0.667718,0.477532,0.798531,0.664556,1.0,0.0,0.445161,0.270455,0.588686,0.569565,...,0.268586,0.151954,0.301142,0.238393,0.217568,0.246487,0.136518,0.304188,0.464286,0.367164
9634,0.493763,0.136728,0.890305,0.745508,1.0,0.0,0.461290,0.295455,0.589051,0.443478,...,0.272716,0.165517,0.277030,0.113839,0.165315,0.331382,0.131937,0.353403,0.539286,0.528358
7950,0.185214,0.492611,0.345933,0.423138,1.0,0.0,0.430108,0.204545,0.616180,0.478261,...,0.247393,0.138697,0.179357,0.096726,0.165916,0.430913,0.190881,0.300175,0.601190,0.609453


In [9]:
from sklearn.linear_model import LogisticRegression 

# Logistic Regression with max_iter=200 
log_model = LogisticRegression(max_iter=200, verbose=2, random_state=42)
log_model.fit(x_train_scaled, y_train_scaled)
y_pred_log = log_model.predict(x_test_scaled)
print(metrics.accuracy_score(y_test_scaled, y_pred_log))

0.6571165926152903


From our previous tests, we found that logistic regression gave around 65% accuracy. Let us try to find the optimal parameters.

In [10]:
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     'C': [0.001, 0.01, 0.1, 0.5, 0.75, 1, 1.25, 1.5, 2, 5, 10, 100],
#     'penalty': ['l1', 'l2', 'elasticnet', 'none'],
#     'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
# }

# log_model = LogisticRegression(max_iter=1000)
# clf = GridSearchCV(log_model, param_grid, cv=5, scoring='accuracy', verbose=3)

In [11]:
# clf.fit(x_train_scaled, y_train_scaled)

In [12]:
# print("Best Parameters:", clf.best_params_)

### Train using best parameters

We have found "optimal" parameters with GridSearchCV:

Best Parameters: {'C': 0.01, 'penalty': 'l2', 'solver': 'liblinear'}

Let's evaluate the model using these parameters.

In [13]:
opt_log_model = LogisticRegression(max_iter=1000, verbose=2, random_state=42, C=0.01, penalty='l2', solver='liblinear')
opt_log_model.fit(x_train_scaled, y_train_scaled)
y_pred = opt_log_model.predict(x_test_scaled)
print(metrics.accuracy_score(y_test_scaled, y_pred))

[LibLinear]0.6557377049180327


It seems like the parameters do not give much of a performance bump. Let's just use the LogisticRegression with max_iter=1000 and default params.

What we can try to do is a [k-fold cross validation](https://www.youtube.com/watch?v=kituDjzXwfE&t=698s). Let's see if training the model with different k's using cross validation will improve performance.

In [14]:
## Cross Validation on Logistic Regression Model